In [ ]:
pip install transformers

In [2]:
import sys
sys.path.append('..')

In [ ]:
from ZeroShot_Baseline import QAModel
from transformers import AutoTokenizer, OPTForCausalLM, GPT2Tokenizer, GPT2LMHeadModel
from helper import *
import torch
import numpy as np
from sklearn.utils import shuffle
import json
import pandas as pd

In [ ]:
config = load_config()
train_splits, val_splits, test_splits = load_data(config)

In [ ]:
def ZeroShot_QA(model, data):
	correct = 0
	type_correct = {
		"Specific": 0,
		#"Subjective": 0,
		"Commonsense": 0,
		#"Compound": 0,
		"Negated": 0,
		"Analogical": 0,
		"Temporal": 0}
	type_count = {
		"Specific": 0,
		#"Subjective": 0,
		"Commonsense": 0,
		#"Compound": 0,
		"Negated": 0,
		"Analogical": 0,
		"Temporal": 0}

	for sample in data:
		for key in sample['query_type']:
			if sample['query_type'][key] == 1:
				type_count[key] += 1
		options_list = [val for val in sample['options'].values()]
		query = sample['query']
		correct_answer = sample['options'][sample['answer']]
		answer = model.get_answer(query, options_list)

		if answer == correct_answer:
			correct += 1
			for key in sample['query_type']:
				if sample['query_type'][key] == 1:
					type_correct[key] += 1

	return correct, len(data), type_correct, type_count

In [ ]:
def evaluate(name, model_name):
    model_config = {
		"facebook/opt-1.3b": [OPTForCausalLM, AutoTokenizer], 
		"gpt2": [GPT2LMHeadModel, GPT2Tokenizer]
	}
    model = model_config[model_name][0]
    tokenizer = model_config[model_name][1]
    
    results = []
    for i in range(len(train_splits)):
        test_data = test_splits[i]
        qa_model = QAModel(model, tokenizer, model_name)
        correct, total, type_correct, type_count = ZeroShot_QA(qa_model, test_data)

        for key, val in type_correct.items():
            type_correct[key] = val*100/type_count[key]
        type_correct.update({"All":correct*100/total})
        results.append(type_correct)

    df = pd.DataFrame(results)
    df.to_csv("ZeroShot_"+name+".csv")

In [ ]:
evaluate("OPT-1.3b", "facebook/opt-1.3b")
evaluate("GPT-2", "gpt-2")